In [ ]:
import os, sys
from collections import defaultdict
from tqdm import tqdm

sys.path.append("../")

import random
import numpy as np
import torch
import learn2learn as l2l

from src.datasets.fastspeech_dataset import (
    build_path_to_transcript_dict_libri_tts,
    FastSpeechDataset,
)
from src.datasets.meta_fastspeech_dataset import MetaFastSpeechDataset
from src.tts.models.fastspeech2.FastSpeech2 import FastSpeech2
from src.pipelines.gst_fastspeech2 import meta_train_loop, train_loop

device = "cuda" if torch.cuda.is_available() else "cpu"
device

In [ ]:
TEST_CLEAN_PATH = "../data/test-clean"
ALIGNER_CHECKPOINT = "../saved_models/aligner.pt"

In [ ]:
transcript_dict = build_path_to_transcript_dict_libri_tts(TEST_CLEAN_PATH)

dataset = MetaFastSpeechDataset(
    path_to_transcript_dict=transcript_dict,
    acoustic_checkpoint_path=ALIGNER_CHECKPOINT,  # path to aligner.pt
    cache_dir="./librispeech/",
    lang="en",
    loading_processes=2,  # depended on how many CPU you have
    device=device,
    n_way_shot=1,
    n_way_query=1,
    k_shot=1,
    k_query=1,
    n_task=1,
    n_batch=1,
)

In [ ]:
meta_train_loop.train_loop(
    net,
    dataset,
    device=device,
    save_directory="./save_models",
    path_to_checkpoint=None,
    resume=False,
    # fine_tune=True,
    phase_1_steps=2,
    phase_2_steps=2,
    adaptation_steps=1,
    steps_per_save=1,
    path_to_embed_model="../saved_models/embedding_function.pt",
    inner_lr=0.01,
    outer_lr=0.001,
    warmup_steps=1,
    use_wandb=False,
)